In [ ]:

from google.colab import drive
drive.flush_and_unmount()  # unmount
drive.mount('/content/drive')  # mount again

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
import os

dir = '/content/drive/MyDrive/tern_project/Eyal/FinalResults/'

os.chdir(dir)

In [ ]:
import configparser

# Initialize the ConfigParser object
config = configparser.ConfigParser()
# Read the INI file
config.read('report_breeding_terns.ini')
# A date to count terns on
date = config.get('General', 'date')
# Path to the output directory for multi-scan tracking results
breeding_tracks_dir = config.get('General', 'breeding_tracks_dir')
# Path to final breeding tracks result
final_report_dir = config.get('General', 'final_report_dir')
# Path to overlap areas details
overlap_areas = config.get('General', 'overlap_areas')

In [ ]:
date

'2025_05_29'

### Load breeding tracks and determine classification

collect JSON files list of breeding tracks

In [ ]:
import glob


flags_dirs = [os.path.join(breeding_tracks_dir, dir_name) for dir_name in os.listdir(breeding_tracks_dir) if date in dir_name]

# Collect all JSON files
flags_reports_lists = []
for dir_path in flags_dirs:
    # Get all immediate subdirectories
    subdirs = [os.path.join(dir_path, d) for d in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, d))]

    # Search for JSON files in each subdirectory
    for subdir in subdirs:
        flags_reports_lists.extend(glob.glob(os.path.join(subdir, '*.json')))

Create Dataframe of all breeding track

In [ ]:
import re

def get_flag_number(file_name):
    # Define a regular expression pattern to match the flag number
    pattern = r'flag(\d+).'
    # Use re.search to find the pattern in the file name
    match = re.search(pattern, file_name)
    # Check if a match is found
    if match:
        # Extract the flag number from the matched group
        return match.group(1)
    else:
        print("No flag number found in the file name.")

In [ ]:
from collections import Counter

def select_class(classifier_results):
    # Count the frequency of each class
    class_counts = Counter([result[0] for result in classifier_results])
    max_count = max(class_counts.values())

    # Find classes with the maximum frequency
    most_frequent_classes = [cls for cls, count in class_counts.items() if count == max_count]

    if len(most_frequent_classes) == 1:
        # If only one class is most frequent, return it
        selected_class = most_frequent_classes[0]
        conf = sum([result[1] for result in classifier_results if result[0] == selected_class]) / max_count
    else:
        cls_conf = {}
        for result in classifier_results:
            for cls in most_frequent_classes:
                if cls == result[0]:
                    cls_conf[cls] = cls_conf.get(cls, 0) + result[1]
                    break

        selected_class = max(cls_conf, key=cls_conf.get)
        conf = cls_conf[selected_class] / max_count

    return selected_class, conf

In [ ]:
import json

breeding_tracks = []

for flag_file in flags_reports_lists:
    with open(flag_file, 'r') as file:
        tracks_details = json.load(file)

    # Extract flag number and tern classes distribution
    flag_number = get_flag_number(flag_file)


    # Process each nest detail
    for track in tracks_details['nests_details']:
        pred_class, conf = select_class(track['track_classes'])
        relevant_data = {
            **track['average_location'],
            'predict_class': pred_class,
            'confidence': conf,
            'flag': flag_number,
        }
        breeding_tracks.append(relevant_data)

In [ ]:
import pandas as pd

# Create dataframe for store boxes details
breeding_tracks_df = pd.DataFrame(breeding_tracks)

In [ ]:
breeding_tracks_df.head()

,x1,x2,y1,y2,predict_class,confidence,flag
0,1006.576860,1070.980881,542.830903,576.325667,Common-Sit,0.899000,92
1,888.565329,943.118272,648.280055,690.086947,Common-Sit,0.804000,92
2,946.031245,996.275793,443.766410,479.475758,Common-Sit,0.671333,92
3,533.818526,583.828121,319.587843,346.384425,Common-Stand,0.952500,92
4,1224.113788,1270.380390,479.120651,518.052274,Common-Sit,0.512500,92


Calculating for each box the width and heigh size in cm. The calculations involve camera calibration, 3D to 2D transformations, and the use of PTZ (Pan-Tilt-Zoom) camera parameters. It's done by using a library writted to mapping box pixel coordinates into real world location.

In [ ]:
import sys

sys.path.append('/content/drive/MyDrive/tern_project/Eyal/RealCoordinatesCalculator')
from real_coordinates_calculator import RealCoordinatesCalculator


real_coordinates_calculator = RealCoordinatesCalculator()
# Group by 'flag' and apply the box size calculation function
boxes_size_df = breeding_tracks_df.groupby('flag').apply(real_coordinates_calculator.calc_box_size)

# Drop flag as index from dataframe
boxes_size_df.index = boxes_size_df.index.droplevel(0)

# Merge the calculated results back into the original DataFrame
breeding_tracks_df = pd.merge(breeding_tracks_df, boxes_size_df, left_index=True, right_index=True)

# Display the resulting DataFrame
breeding_tracks_df.head()

Loading file /content/drive/MyDrive/tern_project/Eyal/RealCoordinatesCalculator/PTZCamValues181_mod.txt...
Loading file /content/drive/MyDrive/tern_project/Eyal/RealCoordinatesCalculator/PTZCamValues191_mod.txt...


/tmp/ipython-input-11-419568255.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  boxes_size_df = breeding_tracks_df.groupby('flag').apply(real_coordinates_calculator.calc_box_size)


,x1,x2,y1,y2,predict_class,confidence,flag,tern_x,tern_y,dy/dx_uv,pix_x1,pix_y1,pix_x2,pix_y2,DX_cm,DY_cm,dx_pix_drone,dy_pix_drone,Area
0,1006.576860,1070.980881,542.830903,576.325667,Common-Sit,0.899000,92,1038.778870,576.325667,0.520073,10456,17340,10524,17191,37.728885,19.621758,68,149,740.307051
1,888.565329,943.118272,648.280055,690.086947,Common-Sit,0.804000,92,915.841801,690.086947,0.766354,10777,16981,10850,16834,40.653132,31.154710,73,147,1266.536520
2,946.031245,996.275793,443.766410,479.475758,Common-Sit,0.671333,92,971.153519,479.475758,0.710711,10162,17812,10263,17621,55.849698,39.692989,101,191,2216.841437
3,533.818526,583.828121,319.587843,346.384425,Common-Stand,0.952500,92,558.823323,346.384425,0.535829,9867,18707,9960,18503,51.511741,27.601475,93,204,1421.800022
4,1224.113788,1270.380390,479.120651,518.052274,Common-Sit,0.512500,92,1247.247089,518.052274,0.841463,10158,17555,10267,17369,60.501036,50.909370,109,186,3080.069613


### Find all terns pairs representations of same tern and remove the one with less prediction confidence. Thus pairs duplication appear because that Yolo often detects the same tern with two boxes- one for each class.

In [ ]:
def calc_iou_box_vs_box(box1, box2):
        x1, x1_max, y1, y1_max = box1['x1'], box1['x2'], box1['y1'], box1['y2']
        x2, x2_max, y2, y2_max = box2['x1'], box2['x2'], box2['y1'], box2['y2']

        # Calculating box1 width and height
        w1 = x1_max - x1
        h1 = y1_max - y1

        # Calculating box1 width and height
        w2 = x2_max - x2
        h2 = y2_max - y2

        # Calculating the coordinates of the intersection rectangle
        x_intersection = max(x1, x2)
        y_intersection = max(y1, y2)
        x_intersection_max = min(x1_max, x2_max)
        y_intersection_max = min(y1_max, y2_max)

        # Calculating the area of intersection
        intersection_area = max(0, x_intersection_max - x_intersection) * max(0, y_intersection_max - y_intersection)

        # Calculating the area of union
        box1_area = w1 * h1
        box2_area = w2 * h2
        union_area = box1_area + box2_area - intersection_area

        # Calculating the IoU
        iou = intersection_area / union_area

        return iou

In [ ]:
def calculate_iou(box1, box2):
    # Extract coordinates of the bounding boxes
    x1_tl, y1_tl, x1_br, y1_br = box1['x1'], box1['x2'], box1['y1'], box1['y2']
    x2_tl, y2_tl, x2_br, y2_br = box2['x1'], box2['x2'], box2['y1'], box2['y2']
    # print(x1_tl, y1_tl, x1_br, y1_br)
    # Calculate the coordinates of the intersection rectangle
    x_intersection_tl = max(x1_tl, x2_tl)
    y_intersection_tl = max(y1_tl, y2_tl)
    x_intersection_br = min(x1_br, x2_br)
    y_intersection_br = min(y1_br, y2_br)

    # Calculate area of intersection rectangle
    intersection_area = max(0, x_intersection_br - x_intersection_tl + 1) * max(0, y_intersection_br - y_intersection_tl + 1)

    # Calculate area of each bounding box
    area_box1 = (x1_br - x1_tl + 1) * (y1_br - y1_tl + 1)
    area_box2 = (x2_br - x2_tl + 1) * (y2_br - y2_tl + 1)

    # Calculate area of union
    union_area = area_box1 + area_box2 - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area

    return iou

In [ ]:
import math

def calc_distance_cm(tern1, tern2):
    i_x_average = (tern1['pix_x2'] + tern1['pix_x1']) / 2
    i_y_average = tern1['pix_y1']
    j_x_average = (tern2['pix_x2'] + tern2['pix_x1']) / 2
    j_y_average = tern2['pix_y1']

    return math.sqrt(pow(abs(i_y_average - j_y_average), 2) + pow(abs(i_x_average - j_x_average), 2))

def find_duplicate_terns(sub_df, threshold=0.5):
    """
    Find all pairs of rectangles with IoU greater than the threshold.
    """
    duplicate_pairs = []
    flag = sub_df.iloc[0]['flag']
    for i in range(len(sub_df)):
        for j in range(i + 1, len(sub_df)):
            iou = calc_iou_box_vs_box(sub_df.iloc[i], sub_df.iloc[j])
            if iou > threshold:
                # distance = calc_distance_cm(sub_df.iloc[i], sub_df.iloc[j])
                duplicate_pairs.append((i, j))
                # duplicate_pairs.append((flag, i, j, iou, distance))
    return duplicate_pairs

In [ ]:
duplicate_pairs = breeding_tracks_df.groupby('flag').apply(find_duplicate_terns)

/tmp/ipython-input-15-3116463615.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  duplicate_pairs = breeding_tracks_df.groupby('flag').apply(find_duplicate_terns)


Remove one of each duplicate pair of the same tern. We keep the one tern with higher prediction probability.

In [ ]:
duplicates_to_remove = []

for flag, pairs in duplicate_pairs.items():
    if len(pairs) == 0:
        continue

    # get index of first row of current flag
    flag_indx = breeding_tracks_df[breeding_tracks_df['flag'] == flag].index[0]

    for pair in pairs:
        confidence1 = breeding_tracks_df.iloc[flag_indx + pair[0]]['confidence']
        confidence2 = breeding_tracks_df.iloc[flag_indx + pair[1]]['confidence']
        duplicate_tern_indx = flag_indx + (pair[0] if confidence1 > confidence2 else pair[1])
        duplicates_to_remove.append(duplicate_tern_indx)


# Remove duplicate terns based on indexes
breeding_tracks_df = breeding_tracks_df.drop(duplicates_to_remove)
breeding_tracks_df = breeding_tracks_df.reset_index(drop=True)

print(f'{len(duplicates_to_remove)} duplicate terns detected and removed on the same flag')

45 duplicate terns detected and removed on the same flag


### Remove duplicate terns from different flags due to flags overlays.

In [ ]:
import json

# Read the JSON file
with open(overlap_areas, 'r') as json_file:
    redundant_areas = json.load(json_file)

In [ ]:
def is_point_in_any_polygon(point, polygons):
    for polygon_coords in polygons:
        polygon = Polygon(polygon_coords)
        if polygon.contains(point):
            return True
    return False

In [ ]:
from google.colab.patches import cv2_imshow

import cv2

## The drone image
drone_img = cv2.imread('/content/drive/MyDrive/tern_project/drone_rotem/Shafiyot_True_Ortho.tif')

In [ ]:
from shapely.geometry import Polygon
import numpy as np


for flag in redundant_areas.keys():

    polygons = redundant_areas[flag]

    for polygon in polygons:
        p = Polygon(polygon)

        # Extracting exterior coordinates of the polygon
        exterior_coords = np.array(list(p.exterior.coords), dtype=np.int32)

        cv2.polylines(drone_img, [exterior_coords], isClosed=True, color=(255, 255, 255), thickness=7)

In [ ]:
from shapely.geometry import Point, Polygon


# Collect indices of rows to remove
indices_to_remove = []

for index, track in breeding_tracks_df.iterrows():
    if(track['flag'] not in redundant_areas):
        continue

    center = Point((track.loc['pix_x1'] + track.loc['pix_x2']) / 2, (track.loc['pix_y1'] + track.loc['pix_y2']) / 2)
    if is_point_in_any_polygon(center, redundant_areas[track['flag']]):
        indices_to_remove.append(index)


In [ ]:
from google.colab.patches import cv2_imshow
import cv2


for indx in indices_to_remove:
    (x_center, y_center) = ((breeding_tracks_df.iloc[indx]['pix_x1'] + breeding_tracks_df.iloc[indx]['pix_x2']) / 2,
            (breeding_tracks_df.iloc[indx]['pix_y1'] + breeding_tracks_df.iloc[indx]['pix_y2']) / 2)

    cv2.circle(drone_img, [int(x_center) , int(y_center)], radius=25, color=(0,0,255), thickness=-1)


# Resize the image
scale_factor = 0.1
resized_image = cv2.resize(drone_img, (0, 0), fx=scale_factor, fy=scale_factor)

cv2_imshow(resized_image)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
breeding_tracks_df = breeding_tracks_df.drop(index=indices_to_remove)
breeding_tracks_df = breeding_tracks_df.reset_index(drop=True)

print(f'{len(indices_to_remove)} duplicate terns detected and removed from the different flags')

312 duplicate terns detected and removed from the different flags


Shelve terns that are detected as Standing or Background

In [ ]:
breeding_tracks_df = breeding_tracks_df[breeding_tracks_df['predict_class'].str.contains('Sit', na=False)]

#### Display the terns locations on the drone map

In [ ]:
import cv2

## The drone image
drone_img = cv2.imread('/content/drive/MyDrive/tern_project/drone_rotem/Shafiyot_True_Ortho.tif')

In [ ]:
from google.colab.patches import cv2_imshow



for index, row in breeding_tracks_df.iterrows():
    if 'Common' in row['predict_class']:
        cv2.circle(drone_img, [row['pix_x1'], row['pix_y1']], radius=25, color=(0,0,255), thickness=-1)
    elif 'Little' in row['predict_class']:
        cv2.circle(drone_img, [row['pix_x1'], row['pix_y1']], radius=25, color=(0,255,255), thickness=-1)
    else:
        cv2.circle(drone_img, [row['pix_x1'], row['pix_y1']], radius=25, color=(255,0,0), thickness=-1)


# Resize the image
scale_factor = 0.1
resized_image = cv2.resize(drone_img, (0, 0), fx=scale_factor, fy=scale_factor)

cv2_imshow(resized_image)

Output hidden; open in https://colab.research.google.com to view.

Save simple report of breeding terns nunmber

In [ ]:
from pathlib import Path
from collections import Counter


breeding_tracks_counter = Counter(breeding_tracks_df['predict_class'])

# Define the file path where you want to save the results
output_file = Path(f"BreedingReports/2025/{date}/report.txt")

# Ensure the parent directory exists
output_file.parent.mkdir(parents=True, exist_ok=True)

# Create the breeding_tracks_counter
breeding_tracks_counter = Counter(breeding_tracks_df['predict_class'])

# Prepare the output string
output_lines = [f"Terns on colony on {date}:\n"]
for category, count in breeding_tracks_counter.items():
    output_lines.append(f"{category}: {count}\n")

# Save the output to the file
with output_file.open("w") as f:
    f.writelines(output_lines)

# Optionally, also print the output to the console
print("".join(output_lines))


Terns on colony on 2025_05_29:
Common-Sit: 1307
Little-Sit: 140



In [ ]:
from pathlib import Path

breeding_tracks_counter = Counter(breeding_tracks_df['predict_class'])

# Define the file path where you want to save the results
output_file = Path(f"BreedingReports/2025/{date}/report.txt")

# Create the breeding_tracks_counter
breeding_tracks_counter = Counter(breeding_tracks_df['predict_class'])

# Prepare the output string
output_lines = [f"Terns on colony on {date}:\n"]
for category, count in breeding_tracks_counter.items():
    output_lines.append(f"{category}: {count}\n")

# Save the output to the file
with output_file.open("w") as f:
    f.writelines(output_lines)

# Optionally, also print the output to the console
print("".join(output_lines))


Terns on colony on 2025_05_29:
Common-Sit: 1307
Little-Sit: 140



### Save breeding terns report

Save images with detected breeding terns

In [ ]:
def get_camera_dir(images_dir, date, cameras_num):
    paths = [os.path.join(images_dir, dir_name) for dir_name in os.listdir(images_dir) if date in dir_name]

    images_date_dirs = {
        camera: None
        for camera in cameras_num
    }
    for camera in cameras_num:
        for path in paths:
            if camera in path:
                images_date_dirs[camera] = {'path': f'{path}/tour0', 'dir_name': os.path.basename(path)}
                break
    return images_date_dirs

In [ ]:
images_dir = '/content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/2025'

cameras_names = ['181', '191']
images_date_dirs = get_camera_dir(images_dir, date, cameras_names)

images_date_dirs

{'181': {'path': '/content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/2025/atlitcam181.stream_2025_05_29_10_01_50/tour0',
  'dir_name': 'atlitcam181.stream_2025_05_29_10_01_50'},
 '191': {'path': '/content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/2025/atlitcam191.stream_2025_05_29_13_59_50/tour0',
  'dir_name': 'atlitcam191.stream_2025_05_29_13_59_50'}}

Save JSONs files including terns coordinates and classification

In [ ]:
import numpy as np


def draw_terns_on_image(image_file, track_terns):
    image = cv2.imdecode(np.fromfile(image_file, dtype=np.uint8), cv2.IMREAD_UNCHANGED)

    for _, track_tern in track_terns.iterrows():
        start_point = (int(track_tern['x1']), int(track_tern['y1']))
        end_point = (int(track_tern['x2']), int(track_tern['y2']))


        color = (255,0,0)
        text = 'Other'

        if 'Common-Sit' in track_tern['predict_class']:
            color = (0, 0, 255)
            text = 'Common'
        elif 'Little-Sit' in track_tern['predict_class']:
            color = (0, 255, 255)
            text = 'Little'

        cv2.rectangle(image, start_point, end_point, color, thickness = 2)

        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        font_thickness = 1

        cv2.putText(image, text, (int(track_tern['x1']), int(track_tern['y1']) - 5), font, font_scale, color, font_thickness, cv2.LINE_AA)

    cv2.imwrite(image_file, image)

In [ ]:
import shutil


# Ensure the directory exists
os.makedirs(f"BreedingReports/{date}", exist_ok=True)

print(f"Saving report on {date}..")

terns_counter = {flag: 0 for flag in breeding_tracks_df['flag'].unique()}
# Group by 'flag' and generate JSON files
for flag_num, tracks in breeding_tracks_df.groupby("flag"):
    date_dir = f"{final_report_dir}/{date}"
    report = tracks[["pix_x1", "pix_y1", "predict_class"]].to_dict(orient="records")
    file_path = f"{date_dir}/flag{flag_num}.json"

    with open(file_path, "w") as json_file:
        json.dump(report, json_file, indent=4)


    images_path, dir_name, camera_num = None, None, None
    for camera_name in cameras_names:
        if os.path.isfile(f'{images_date_dirs[camera_name]["path"]}/flag{flag_num}_1_{images_date_dirs[camera_name]["dir_name"]}.jpg'):
            images_path = f'{images_date_dirs[camera_name]["path"]}/flag{flag_num}_1_{images_date_dirs[camera_name]["dir_name"]}.jpg'
            dir_name = images_date_dirs[camera_name]["dir_name"]
            camera_num = camera_name
            break
    if not images_path:
        print(f"No images found for flag {flag_num}")
        continue

    source_file, target_file = images_path, f'{date_dir}/flag{flag_num}.jpg'
    try:
        shutil.copy2(source_file, target_file)
        draw_terns_on_image(target_file, tracks)
    except Exception as e:
        print(f"Failed to copy image for flag '{flag_num}': {e}")

print(f"Reports files are saved...")

Saving report on 2025_05_29..
Reports files are saved...


Save drone image of colony with all breeding terns locations

In [ ]:
# Resize the image
scale_factor = 0.1
resized_image = cv2.resize(drone_img, (0, 0), fx=scale_factor, fy=scale_factor)

# Save the resized image
save_path = os.path.join(date_dir, "terns_locations.jpg")
cv2.imwrite(save_path, resized_image)
print(f"Drone image saved at: {save_path}")

Drone image saved at: /content/drive/MyDrive/tern_project/Eyal/FinalResults/BreedingReports/2025/2025_05_29/terns_locations.jpg
